<a href="https://colab.research.google.com/github/IngwarHighlander/Data-Analytics-Public/blob/main/AB_Testing_Tool_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A/B Testing Tool Results**


**Tableau:**
https://public.tableau.com/app/profile/igor.gutsulyak/viz/ABTest_17482096615350/ABTestingTool?publish=yes

In [ ]:
# Installation of the library to work with Google BigQuery:
!pip install --upgrade google-cloud-bigquery

In [ ]:
# Import of modules for authentication and working with BigQuery and libraries for processing query results:
from google.colab import auth
from google.cloud import bigquery
from google.colab import drive
drive.mount("/content/drive")

import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm


Mounted at /content/drive


In [ ]:
# Authentication to work with Google Cloud:
auth.authenticate_user()

In [ ]:
# Creation of a client to work with BigQuery:
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
# SQL query
query = """
WITH session_info AS (
  SELECT
        s.date,
        s.ga_session_id,
        sp.country,
        sp.device,
        sp.continent,
        sp.channel,
        ab.test,
        ab.test_group
  FROM `DA.ab_test` ab
  JOIN `DA.session` s
  ON ab.ga_session_id = s.ga_session_id
  JOIN `DA.session_params` sp
  ON sp.ga_session_id = ab.ga_session_id
),
session_with_orders AS (
  SELECT
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        COUNT(DISTINCT o.ga_session_id) AS session_with_orders
  FROM `DA.order` o
  JOIN session_info
  ON o.ga_session_id = session_info.ga_session_id
  GROUP BY
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group
),
events AS (
  SELECT
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        ep.event_name,
        COUNT(ep.ga_session_id) AS event_cnt
  FROM `DA.event_params` ep
  JOIN session_info
  ON ep.ga_session_id = session_info.ga_session_id
  GROUP BY
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        ep.event_name
),
session AS (
  SELECT
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
  FROM session_info
  GROUP BY
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group
),
account AS (
  SELECT
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
  FROM `DA.account_session` acs
  JOIN session_info
  ON acs.ga_session_id = session_info.ga_session_id
  GROUP BY
        session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group
)
SELECT
        session_with_orders.date,
        session_with_orders.country,
        session_with_orders.device,
        session_with_orders.continent,
        session_with_orders.channel,
        session_with_orders.test,
        session_with_orders.test_group,
        'session with orders' AS event_name,
        session_with_orders.session_with_orders AS value
FROM session_with_orders
UNION ALL
SELECT
        events.date,
        events.country,
        events.device,
        events.continent,
        events.channel,
        events.test,
        events.test_group,
        events.event_name,
        events.event_cnt AS value
FROM events
UNION ALL
SELECT
        session.date,
        session.country,
        session.device,
        session.continent,
        session.channel,
        session.test,
        session.test_group,
        'session' AS event_name,
        session.session_cnt AS value
FROM session
UNION ALL
SELECT
        account.date,
        account.country,
        account.device,
        account.continent,
        account.channel,
        account.test,
        account.test_group,
        'new_account' AS event_name,
        account.new_account_cnt AS value
FROM account
"""

# Execution of SQL query
query_job = client.query(query)
# Waiting for the request to complete
results = query_job.result()
# Outputting results
df = results.to_dataframe()
df

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,Paraguay,desktop,Americas,Organic Search,2,1,session with orders,1
1,2020-11-03,Ecuador,mobile,Americas,Direct,2,2,session with orders,1
2,2020-11-04,Slovenia,desktop,Europe,Organic Search,2,2,session with orders,1
3,2020-11-05,Malta,mobile,Europe,Paid Search,2,1,session with orders,1
4,2020-11-06,Croatia,mobile,Europe,Direct,2,2,session with orders,1
...,...,...,...,...,...,...,...,...,...
800991,2020-11-05,Vietnam,desktop,Asia,Direct,1,2,first_visit,1
800992,2020-11-08,Vietnam,desktop,Asia,Undefined,2,1,session_start,1
800993,2020-11-09,Vietnam,mobile,Asia,Direct,1,2,page_view,1
800994,2020-12-17,Vietnam,desktop,Asia,Organic Search,3,1,session_start,1


## **Overal description of the dataset**

In [ ]:
# number of rows and columns:
print(f'Number of rows: {df.shape[0]}')
print(f'Number of columns: {df.shape[1]}')

# type of rows and their counts:
print(df.dtypes)
print('\n')
print(df.dtypes.value_counts())


Number of rows: 800996
Number of columns: 9
date          dbdate
country       object
device        object
continent     object
channel       object
test           Int64
test_group     Int64
event_name    object
value          Int64
dtype: object


object    5
Int64     3
dbdate    1
Name: count, dtype: int64


In [ ]:
# Convert 'date' to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [ ]:
# defining common placeholders representing missing values
missing_placeholders = ['(not set)', 'None', '<NA>', '(data deleted)', 'NaN']

df_with_na = df.replace(missing_placeholders, np.nan)

missing_values = df_with_na.isna().sum()
total_missing = missing_values.sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

print("Missing values:")
if not missing_values.empty:
    print(f"\n   Total number of missing values: {total_missing}")
    print(f"\nMissing values by rows:\n{missing_values.to_string()}")
else:
    print("   There are no missing values.")

Missing values:

   Total number of missing values: 28063

Missing values by rows:
country      22063
continent     6000


## **A/B Testing**

### **General Conversion Metrics**

In [ ]:
# Define key conversion metrics (numerator / denominator)
metrics = {
    'add_payment_info / session': ('add_payment_info', 'session'),
    'add_shipping_info / session': ('add_shipping_info', 'session'),
    'begin_checkout / session': ('begin_checkout', 'session'),
    'new_account / session': ('new_account', 'session')
}

results = []

# Loop through each test number (total test level)
for test_number in df['test'].unique():

    # Filter the dataset
    subset = df[df['test'] == test_number]

    # Loop through each defined conversion metric
    for metric_name, (numerator_event, denominator_event) in metrics.items():

        # Aggregate numerator and denominator counts for test group
        numerator_test = subset[(subset['event_name'] == numerator_event) & (subset['test_group'] == 2)]['value'].sum()
        denominator_test = subset[(subset['event_name'] == denominator_event) & (subset['test_group'] == 2)]['value'].sum()

        # Aggregate numerator and denominator counts for control group
        numerator_control = subset[(subset['event_name'] == numerator_event) & (subset['test_group'] == 1)]['value'].sum()
        denominator_control = subset[(subset['event_name'] == denominator_event) & (subset['test_group'] == 1)]['value'].sum()

        # Proceed only if denominators are valid
        if denominator_test > 0 and denominator_control > 0:
            # Calculate conversion rates
            rate_test = numerator_test / denominator_test
            rate_control = numerator_control / denominator_control
            change_pct = (rate_test - rate_control) / rate_control * 100

            # Perform z-test for proportions
            z_stat, p_value = sm.stats.proportions_ztest(
                [numerator_test, numerator_control],
                [denominator_test, denominator_control]
            )

            # SAppend results to list
            results.append({
                "test_number": test_number,
                "metric": metric_name,
                "numerator_event": numerator_event,
                "denominator_event": denominator_event,
                "numerator_count_test": numerator_test,
                "denominator_count_test": denominator_test,
                "conversion_rate_test": rate_test,
                "numerator_count_control": numerator_control,
                "denominator_count_control": denominator_control,
                "conversion_rate_control": rate_control,
                "metric_change_%": change_pct,
                "z_stat": z_stat,
                "p_value": p_value,
                "significant": p_value < 0.05
            })

# Convert list of results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV
results_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/Mate_Academy/ab_test_portfolio_project2_general.csv', index=False)

# Show results
results_df


,test_number,metric,numerator_event,denominator_event,numerator_count_test,denominator_count_test,conversion_rate_test,numerator_count_control,denominator_count_control,conversion_rate_control,metric_change_%,z_stat,p_value,significant
0,2,add_payment_info / session,add_payment_info,session,2409,50244,0.047946,2344,50637,0.046290,3.576911,1.240994,0.214608,False
1,2,add_shipping_info / session,add_shipping_info,session,3510,50244,0.069859,3480,50637,0.068724,1.650995,0.709557,0.477979,False
2,2,begin_checkout / session,begin_checkout,session,4313,50244,0.085841,4262,50637,0.084168,1.988164,0.952898,0.340642,False
3,2,new_account / session,new_account,session,4184,50244,0.083274,4165,50637,0.082252,1.241934,0.588793,0.556000,False
4,1,add_payment_info / session,add_payment_info,session,2229,45193,0.049322,1988,45362,0.043825,12.542021,3.924884,0.000087,True
5,1,add_shipping_info / session,add_shipping_info,session,3221,45193,0.071272,3034,45362,0.066884,6.560481,2.603571,0.009226,True
6,1,begin_checkout / session,begin_checkout,session,4021,45193,0.088974,3784,45362,0.083418,6.660587,2.978783,0.002894,True
7,1,new_account / session,new_account,session,3681,45193,0.081451,3823,45362,0.084278,-3.354299,-1.542883,0.122859,False
8,4,add_payment_info / session,add_payment_info,session,3601,105141,0.034249,3731,105079,0.035507,-3.541234,-1.571106,0.116158,False
9,4,add_shipping_info / session,add_shipping_info,session,4956,105141,0.047137,5128,105079,0.048801,-3.411125,-1.785795,0.074132,False


### Interpretation of results:

*A/B tests for four conversion metrics across four test campaigns (1–4):*

Each test compares a test group (2) vs. a control group (1) across the conversion metrics. Each row represents: one test number (1–4) and one metric with conversion rates and a z-test to check statistical significance (p_value < 0.05)

1. Test 1 had a positive and statistically significant impact on 3 out of 4 metrics (except for new_account). Conversion funnel improvements were real.

- add_payment_info / sessions:	+12.54%	(p-value = 0.000087);
- add_shipping_info / sessions:	+6.56%	(p-value = 0.0092);
- begin_checkout / sessions:	+6.66%	(p-value = 0.0029);
- new_account	/ sessions: -3.35%	(p-value = 0.123, non-significant).

2. Test 2 showed no statistically significant change. Minor improvements, but not reliable.
- add_payment_info / sessions:	+3.58%	(p-value = 0.214, non-significant);
add_shipping_info / sessions:	+1.65%	(p-value = 0.478, non-significant);
begin_checkout / sessions:	+1.99%	(p-value = 0.341, non-significant);
new_account	/ sessions: +1.24%	(p-value = 0.556, non-significant).

3. Test 3 showed slight mixed effects. Only begin_checkout showed a significant drop, suggesting a negative impact in the test group for that step.

- add_payment_info / sessions:	+1.47%	(p-value = 0.520, non-significant);
- add_shipping_info / sessions:	-2.62%	(p-value = 0.157, non-significant);
- begin_checkout / sessions:	-3.35% (negative)	(p-value = 0.012);
- new_account / sessions:	-1.13%	(p-value = 0.520, non-significant).

4. Test 4 had mixed results with some significance. Two significant negative effects suggest Test 4 may have harmed user progression. Needs further investigation.

- add_payment_info / sessions:	-3.54%	(p-value = 0.116, non-significant);
- add_shipping_info / sessions:	-3.41%	(p-value = 0.074, non-significant);
- begin_checkout / sessions:	-2.35% (negative)	(p-value = 0.046);
- new_account / sessions:	-3.36% (negative)	(p-value = 0.018).

**Overall takeaways:**
- Test 1: Significantly improved key metrics (except for new_account). Likely worth deploying.
- Test 2: No impact detected. Consider reworking test elements.
- Test 3: One significant drop (begin_checkout) — risky.
- Test 4:	Two conversion steps worsened — avoid rollout.

**Suggestions:**
- Double down on ideas from Test 1 – find what worked and scale it.
- Rethink Tests 3 & 4 – isolate what may have caused drop-offs.
- Consider deeper segment analysis (e.g., by device, user cohort) to see if some groups reacted better/worse.

### **Convertion Metrics vs Devices**

In [ ]:
# Define key conversion metrics (numerator / denominator)
metrics = {
    'add_payment_info / session': ('add_payment_info', 'session'),
    'add_shipping_info / session': ('add_shipping_info', 'session'),
    'begin_checkout / session': ('begin_checkout', 'session'),
    'new_account / session': ('new_account', 'session')
}

results_devices = []

# Loop through all test numbers and device types
for test_number in df['test'].unique():
    for device in df['device'].unique():
        # Filter data for a specific test number and device
        subset = df[(df['test'] == test_number) & (df['device'] == device)]

        # Loop through each defined metric
        for metric_name, (numerator_event, denominator_event) in metrics.items():

            # Aggregate numerator and denominator counts for test group
            numerator_test = subset[(subset['event_name'] == numerator_event) & (subset['test_group'] == 2)]['value'].sum()
            denominator_test = subset[(subset['event_name'] == denominator_event) & (subset['test_group'] == 2)]['value'].sum()

            # Aggregate numerator and denominator counts for control group
            numerator_control = subset[(subset['event_name'] == numerator_event) & (subset['test_group'] == 1)]['value'].sum()
            denominator_control = subset[(subset['event_name'] == denominator_event) & (subset['test_group'] == 1)]['value'].sum()

            # Proceed only if both denominators are greater than 0
            if denominator_test > 0 and denominator_control > 0:
                # Calculate conversion rates
                rate_test = numerator_test / denominator_test
                rate_control = numerator_control / denominator_control
                change_pct = (rate_test - rate_control) / rate_control * 100

                # Perform z-test for proportions
                z_stat, p_value = sm.stats.proportions_ztest(
                    [numerator_test, numerator_control],
                    [denominator_test, denominator_control]
                )

                # Append results to list
                results_devices.append({
                    "test_number": test_number,
                    "device": device,
                    "metric": metric_name,
                    "numerator_event": numerator_event,
                    "denominator_event": denominator_event,
                    "numerator_count_test": numerator_test,
                    "denominator_count_test": denominator_test,
                    "conversion_rate_test": rate_test,
                    "numerator_count_control": numerator_control,
                    "denominator_count_control": denominator_control,
                    "conversion_rate_control": rate_control,
                    "metric_change_%": change_pct,
                    "z_stat": z_stat,
                    "p_value": p_value,
                    "significant": p_value < 0.05
                })

# Convert list of results to a DataFrame
results_devices_df = pd.DataFrame(results_devices)

# Export the DataFrame to a CSV file
results_devices_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/Mate_Academy/ab_test_portfolio_project2_devices.csv', index=False)

# Display the results
results_devices_df


,test_number,device,metric,numerator_event,denominator_event,numerator_count_test,denominator_count_test,conversion_rate_test,numerator_count_control,denominator_count_control,conversion_rate_control,metric_change_%,z_stat,p_value,significant
0,2,desktop,add_payment_info / session,add_payment_info,session,1401,29380,0.047686,1314,29497,0.044547,7.045601,1.815587,0.069434,False
1,2,desktop,add_shipping_info / session,add_shipping_info,session,2088,29380,0.071069,1988,29497,0.067397,5.448443,1.755041,0.079252,False
2,2,desktop,begin_checkout / session,begin_checkout,session,2585,29380,0.087985,2403,29497,0.081466,8.002258,2.840291,0.004507,True
3,2,desktop,new_account / session,new_account,session,2385,29380,0.081178,2442,29497,0.082788,-1.945218,-0.712176,0.476356,False
4,2,mobile,add_payment_info / session,add_payment_info,session,961,19756,0.048643,978,20017,0.048858,-0.440088,-0.099562,0.920692,False
5,2,mobile,add_shipping_info / session,add_shipping_info,session,1356,19756,0.068637,1417,20017,0.070790,-3.040624,-0.842754,0.399366,False
6,2,mobile,begin_checkout / session,begin_checkout,session,1656,19756,0.083823,1766,20017,0.088225,-4.989937,-1.565423,0.117484,False
7,2,mobile,new_account / session,new_account,session,1701,19756,0.086100,1623,20017,0.081081,6.190524,1.808493,0.070530,False
8,2,tablet,add_payment_info / session,add_payment_info,session,47,1108,0.042419,52,1123,0.046305,-8.391766,-0.445631,0.655864,False
9,2,tablet,add_shipping_info / session,add_shipping_info,session,66,1108,0.059567,75,1123,0.066785,-10.808664,-0.700617,0.483542,False


### Interpretation of results:

*A/B test results for the four conversion metrics (add_payment_info, add_shipping_info, begin_checkout, new_account), split by test number and device type (desktop, mobile, tablet):*

Each row represents: one test number (1–4); one device (desktop, mobile, tablet); one metric with conversion rates and a z-test to check statistical significance (p_value < 0.05)

1. Key Metric Overview:
- add_payment_info / session:	Only some tests/devices show significant improvements (e.g., test 1 desktop/mobile)
- add_shipping_info / session:	Mostly not significant, small effect sizes
- begin_checkout / session:	Shows most consistent and significant improvements across multiple tests/devices
- new_account / session	Rarely significant, very small differences

2. Detailed Metric Findings:
- **add_payment_info / session:** The test variant had better performance in test 1.
  Significant increase in test 1: Desktop: +11.4%, p = 0.007; Mobile: +17.1%, p = 0.0007; Tablet: +36%, p = 0.0458. Other tests: mostly small or insignificant changes

- **add_shipping_info / session:** Slight benefit in some groups, but generally low impact across devices.
  Test 1 Desktop: +12.2%, p = 0.00033.
  Test 4 Desktop/Tablet: small significant improvements.
  Most other results are non-significant (p > 0.05)

- **begin_checkout / session:** This metric consistently improved in the test group — high potential impact metric.

    Significant in Test 1, Test 3, and Test 4 across multiple devices:

        Test 1 Desktop: +14.3%, p = 0.000003;
        Test 1 Tablet: +33%, p = 0.0149;
        Test 3 Tablet: +26%, p = 0.0046;
        Test 4 Mobile: +4.3%, p = 0.0268.

- **new_account / session:** Test variants did not significantly affect new account creation.

    Test 4 Desktop: Slightly significant increase (+3.8%), p = 0.035;

    All others: not significant (p > 0.1), sometimes even lower in the test group.

3. Final takeaways:
- Prioritize **begin_checkout / session** as a key success indicator.
- Audience	Improvements most effective for desktop and mobile users.
- Test 1 variant likely contained beneficial changes, especially for payment + checkout
